Preamble: Import Libraries, Download Dataset, Preprocess Data

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import h5py


from medmnist import AdrenalMNIST3D

# Or NoduleMNIST3D, AdrenalMNIST3D, etc.
# Change these to match which dataset you've been assigned


train_dataset = AdrenalMNIST3D(split='train', size=28, download=True)
trainx = []
trainy = []

val_dataset = AdrenalMNIST3D(split='val', size=28, download=True)
valx = []
valy = []

for i in range(len(train_dataset)):
    trainx.append(train_dataset[i][0])
    trainy.append(train_dataset[i][1])

for i in range(len(val_dataset)):
    valx.append(val_dataset[i][0])
    valy.append(val_dataset[i][1])

#validation
trainx_tensor = tf.convert_to_tensor(trainx, dtype=tf.float16)
trainx_tensor = np.transpose(trainx_tensor, (0,2,3,4,1))

trainy_tensor = tf.convert_to_tensor(trainy, dtype=tf.float16)



#validation
valx_tensor = tf.convert_to_tensor(valx, dtype=tf.float16)
valx_tensor = np.transpose(valx_tensor, (0,2,3,4,1))

valy_tensor = tf.convert_to_tensor(valy, dtype=tf.float16)


# float16 doesn't run any faster on the 4090s, but it cuts memory usage in half!


model = tf.keras.Sequential(
    layers = [
    tf.keras.layers.Input(shape=(28,28,28,1)),
    tf.keras.layers.Conv3D(16, 3, activation="relu"),
    tf.keras.layers.Conv3D(16, 2, activation="relu"),
    tf.keras.layers.Conv3D(16, 2, activation="relu"),
    tf.keras.layers.Conv3D(16, 2, activation="relu"),
    tf.keras.layers.Conv3D(16, 2, activation="relu"),
    tf.keras.layers.Conv3D(16, 2, activation="relu"),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
    ]
)

checkpoint_folder = "checkpoints"
checkpoint_path = checkpoint_folder + "/epoch_{epoch:03d}.weights.h5"

checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True, 
    save_freq="epoch"
)



model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(trainx_tensor, trainy_tensor, validation_data=(valx_tensor, valy_tensor), epochs=5, batch_size=12, callbacks=[checkpoint_cb])

with open("training_history.txt", "w") as f:
    for key, values in history.history.items():
        f.write(f"{key}: {values}\n")

2025-12-12 21:31:04.992939: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/null/.local/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.0 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/home/null/.local/lib/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.0 at tensorflow/core/framework/tensor.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warni

ModuleNotFoundError: No module named 'pyparsing'